In [1]:
using Revise, HarmonicBalance
using DifferentialEquations
import HarmonicBalance.TimeEvolution: ParameterSweep, ODEProblem

┌ Info: Precompiling HarmonicBalance [e13b9ff6-59c3-11ec-14b1-f3d2cc6c135e]
└ @ Base loading.jl:1423


# Harmonic equations for the parametric oscillator

In [ ]:
@variables γ,λ,F,θ,η,α, ω0, ω,t,T, ψ, x(t)

natural_equation =  d(d(x,t),t) + γ*d(x,t) + ω0^2*(1-λ*cos(2*ω*t+ψ))*x + α * x^3 +η *d(x,t) * x^2
forces =  F*cos(ω*t+θ)
dEOM = HarmonicBalance.DifferentialEquation(natural_equation + forces, x)
HarmonicBalance.add_harmonic!(dEOM, x, ω)
@time averagedEOM = HarmonicBalance.get_harmonic_equations(dEOM, slow_time=T, fast_time=t);

# single sweep

In [ ]:
fixed_parameters = ParameterList(ω0 => 1.0,γ => 1E-2, λ => 5E-2, F => 1E-3,  α => 1.,  η=>0.3, θ => 0, ψ => 0)
sweep=ParameterSweep(ω=>(0.95,0.98), (0, 2E4)) # linearly interpolate between two values at two times
prob=ODEProblem(averagedEOM, fixed_parameters, sweep=sweep, x0=[0.0001;0.0001], timespan=(0, 4E4))

In [ ]:
@time time_soln = solve(prob,saveat=100);
HarmonicBalance.plot(time_soln.t, getindex.(time_soln.u,1).^2 .+ getindex.(time_soln.u,2).^2);

# simultaneous sweep of multiple parameters

In [ ]:
fixed_parameters = ParameterList(ω0 => 1.0,γ => 1E-2,  F => 1E-3,  α => 1.,  η=>0.3, θ => 0, ψ => 0)

# interpolate both parameters
sw_params = [ω=>[0.95;1.0],λ => [5E-2;1E-2]]
sweep=ParameterSweep(sw_params, (0, 2E4))

prob=ODEProblem(averagedEOM, fixed_parameters, sweep=sweep; x0=[0.0001;0.0001], timespan=(0,4E4))

@time time_soln = solve(prob,saveat=100);
HarmonicBalance.plot(time_soln.t, getindex.(time_soln.u,1).^2 .+ getindex.(time_soln.u,2).^2);

# series of sweeps

In [ ]:
# combine two sweeps
sweep1 = ParameterSweep(ω => [0.95, 1.0], (0, 2E4))
sweep2 = ParameterSweep(λ => [0.05, 0.01], (2E4, 4E4))
sweep = sweep1 + sweep2

fixed_parameters = ParameterList(ω0 => 1.0,γ => 1E-2,  F => 1E-3,  α => 1.,  η=>0.3, θ => 0, ψ => 0)

prob=ODEProblem(averagedEOM, fixed_parameters, sweep=sweep; x0=[0.0001;0.0001], timespan=(0, 4E4));

In [ ]:
@time time_soln = solve(prob,saveat=100);
HarmonicBalance.plot(time_soln.t, getindex.(time_soln.u,1).^2 .+ getindex.(time_soln.u,2).^2);

In [ ]:
# check out our swept function
times = 0:4E4
HarmonicBalance.plot(times, sweep[ω].(times));

# Define custom sweep functions

In [ ]:
times = 0:2E4
ωfunc(t)=1-0.05*cos(2*pi*t/times[end])
λfunc(t)=0.05-0.027*sin(2*pi*t/times[end])
sweep_circle=ParameterSweep([ω => ωfunc,λ => λfunc])
# check the swept functions
HarmonicBalance.plot((sweep_circle[ω].(times)),(sweep_circle[λ].(times)));

In [ ]:
fixed_parameters = ParameterList(ω0 => 1.0,γ => 1E-2,  F => 1E-3,  α => 1.,  η=>0.3, θ => 0, ψ => 0)
prob=ODEProblem(averagedEOM, fixed_parameters,sweep=sweep_circle; x0=[0.0001;0.0001], timespan=(0, 2E4))
@time time_soln = solve(prob,saveat=100);
HarmonicBalance.plot(getindex.(time_soln.t, 1), getindex.(time_soln.u,1).^2 .+ getindex.(time_soln.u,2).^2);

# Use steady-state solutions as initial conditions

In [ ]:
fixed_parameters = ParameterList(ω0 => 1.0,γ => 1E-2,  F => 1E-3,  α => 1.,  η=>0.3, θ => 0, ψ => 0, λ=>0.05)
params_range = ParameterRange(ω => LinRange(1.04,1.1,100))
ss_problem = HarmonicBalance.Problem(averagedEOM)
steady_soln = HarmonicBalance.get_steady_states(ss_problem, params_range, fixed_parameters);

In [ ]:
# select a solution and evolve from it
index = 75
s1 = get_single_solution(steady_soln, branch=1, index=1);
s2 = get_single_solution(steady_soln, branch=2, index=1);
s3 = get_single_solution(steady_soln, branch=3, index=1);

sweep = ParameterSweep(ω => (s1[ω], 1.1), (0, 1E4))
function t_solve(s)
    problem = ODEProblem(averagedEOM, steady_solution=s, timespan=(0,1E4), sweep=sweep)
    time_soln = solve(problem,saveat=10);
end

In [ ]:
# time-evolve all three solutions
time_soln_1 = t_solve(s1)
time_soln_2 = t_solve(s2)
time_soln_3 = t_solve(s3);

In [ ]:
# contrast to steady-state solution
plot_td(s) = HarmonicBalance.plot(s.t, sqrt.(getindex.(s.u,1).^2 .+ getindex.(s.u,2).^2))
plot_td(time_soln_1)
plot_td(time_soln_2)
plot_td(time_soln_3)
HarmonicBalance.plot_1D_solutions(steady_soln, x="ω", y="sqrt(u1^2 + v1^2)")